## LSTM Text Classification with Keras

In [ ]:
%run Setup.ipynb

Load embedding matrix into an `Embedding` layer. Toggle `trainable=False` to prevent the weights from being updated during training.

In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

### Structure
[Reference](https://github.com/richliao/textClassifier), [LTSM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm1 = Bidirectional(LSTM(12,dropout=0.3,recurrent_dropout=0.3))(embedded_sequences)
preds = Dense(4, activation='softmax')(l_lstm1)

In [ ]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

In [ ]:
def step_cyclic(epoch):
    try:
        l_r, decay = 1.0, 0.00001
        if epoch%20==0:multiplier = 6
        else:multiplier = 1
        rate = float(multiplier * l_r * 1/(1 + decay * epoch))
        print("Epoch",epoch+1,"- learning_rate",rate)
        return rate
    except Exception as e:
        print("Error in lr_schedule:",str(e))
        return float(1.0)
    
def initial_boost(epoch):
    if epoch==0: return float(6.0)
    else: return float(1.0)
        
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=50, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoints", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#lr_schedule = callbacks.LearningRateScheduler(step_cyclic)
lr_schedule = callbacks.LearningRateScheduler(initial_boost)
fb_message = fb.sendmessage(savelog=True,fexten='TEST',username='tlkh@live.co.uk', password='tlkh!@123')

In [ ]:
model.summary()
print("Training Progress:")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=200, batch_size=50,
          callbacks=[tensorboard, model_checkpoints, lr_schedule])